# Лабораторная работа №3

ФИО: Щедрова Милана Вячеславовна
Группа: БИВТ-21-3

Отправлять можно следующими способами:
1. Запушить этот ноутбук в GitHub в репозиторий, где у вас лежат ноутбуки с лабами

Deadlines:
- Занятие №7 в семестре (Занятие №3 очное)

Что необходимо сделать:  
- Обучить различного рода модели машинного обучения и сравнить их между собой  

---
## Читайте задание внимательно

Исходные данные:
1. В [табличке](https://docs.google.com/spreadsheets/d/11600OkqdRzxc-xZmFTuBs099VeAqU_GlE0Bx6bBlIRE/edit?usp=sharing) необходимо узнать название своего датасета 
2. Скачать нужны вам данные можно в [Google Drive](https://drive.google.com/drive/folders/1Phm-Fq1GL-VX7NS-DCEMRh_Fo1wU7DQ9?usp=sharing)
  
---
Теперь по пунктам, что я от вас жду:  
1. Загрузить необходимые данные к себе и считать (read) их в переменную.
2. Понять, у вас задача классификации (бинарной или многоклассовой) или регрессии (**если у вас многоклассовая классификация, прочтите P.S.S. внизу**).
3. Сделать предобработку данных:  
     1. Разделить выборку на тренировочную (train) и тестовую (test). _Обратите внимание, что обучать скейлеры и определять, какими значениями вы будете заполнять пропуски, вы будете на train выборке, а применять и на train, и на test_.
     2. Проверить пропуски в данных. Если они есть, заполнить одной из стратегий, предложенных в ноутбуке для семинара №3. P.S. Для численных и категориальных переменных будут разные стратегии.
     3. Отнормировать численные переменные (`StandardScaler`, `MinMaxScaler`).
     4. Закодировать категориальные признаки по одной из стратегий.
4. Обучить на тренировочном множестве:
     1. Линейную модель (`LogisticRegression`, `LinearRegression`)
     2. Деревянную модель (`DecisionTreeClassifier`, `DecisionTreeRegressor`) (тут советую попробовать разные глубины деревьев)
     3. K-ближайших соседей (`KNeighborsClassifier`, `KNeighborsRegressor`) (тут тоже есть смысл попробовать разные `k`)
     4. Случайный лес (`RandomForestClassifier`, `RandomForestRegressor`) 
5. Посчитайте метрики на train и test множествах:
     1. Для задачи классификации -- Accuracy, ROC-AUC (график + значение), PR-кривую (график), F1-score
     2. Для задачи регрессии -- MAE, RMSE, MAPE
6. Сравните метрики относительно train/test, так и относительно разных моделей. Ответьте на следующие вопросы:
     1. Какая модель справилась лучше с поставленной задачей?
     2. Имеет ли место переобучение?
     3. Имеет ли место недообучение?
     4. Как можно улучшить метрики моделей?

---
P.S.  
Просьба -- делать каждое задание в отдельных ячейках и с отдельными заголовками (как пункт 1 и 2 в этом ноутбуке) типа  
- Заголовок
- Ячейки с кодом
- Другой заголовок
- Другие ячейки с кодом

P.S.S.  
Если вам повезло с многоклассовой классификацией, вам будет необходимо понять, умеет ли алгоритм работать с несколькими классами одновременно (обычно они не умеют). Поэтому вам может понадобиться такая штука, как OneVsRestClassifier ([ссылка](https://scikit-learn.org/stable/modules/generated/sklearn.multiclass.OneVsRestClassifier.html#sklearn.multiclass.OneVsRestClassifier)), но советую ознакомиться с этой [страницей](https://scikit-learn.org/stable/modules/multiclass.html), здесь представлена более полная информация.

## Импорт библиотек

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from sklearn.preprocessing import OneHotEncoder

## 1. Загрузка данных

In [5]:
data = pd.read_csv('student_data.csv')
data.head()  # если нужно, можно поменять разделитель

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,4,6,10,10


## 2. Тип задачи

Задача регрессии

## 3. Предобработка данных

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 395 entries, 0 to 394
Data columns (total 33 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   school      395 non-null    object
 1   sex         395 non-null    object
 2   age         395 non-null    int64 
 3   address     395 non-null    object
 4   famsize     395 non-null    object
 5   Pstatus     395 non-null    object
 6   Medu        395 non-null    int64 
 7   Fedu        395 non-null    int64 
 8   Mjob        395 non-null    object
 9   Fjob        395 non-null    object
 10  reason      395 non-null    object
 11  guardian    395 non-null    object
 12  traveltime  395 non-null    int64 
 13  studytime   395 non-null    int64 
 14  failures    395 non-null    int64 
 15  schoolsup   395 non-null    object
 16  famsup      395 non-null    object
 17  paid        395 non-null    object
 18  activities  395 non-null    object
 19  nursery     395 non-null    object
 20  higher    

1. Разделить выборку на тренировочную (train) и тестовую (test).

In [7]:
X = data.drop(columns=['G3'])  # Удаляем столбец с метками из данных
y = data['G3']  # Метки для обучения

# Разделяем данные на тренировочный и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
X_train, X_test, y_train, y_test

(    school sex  age address famsize Pstatus  Medu  Fedu      Mjob      Fjob  \
 181     GP   M   16       U     GT3       T     3     3  services     other   
 194     GP   M   16       U     GT3       T     2     3     other     other   
 173     GP   F   16       U     GT3       T     1     3   at_home  services   
 63      GP   F   16       U     GT3       T     4     3   teacher    health   
 253     GP   M   16       R     GT3       T     2     1     other     other   
 ..     ...  ..  ...     ...     ...     ...   ...   ...       ...       ...   
 71      GP   M   15       U     GT3       T     4     2     other     other   
 106     GP   F   15       U     GT3       T     2     2     other     other   
 270     GP   F   19       U     GT3       T     3     3     other  services   
 348     GP   F   17       U     GT3       T     4     3    health     other   
 102     GP   M   15       U     GT3       T     4     4  services     other   
 
      ... romantic famrel  freetime  g

In [9]:
numeric_columns = X.select_dtypes(include=[np.number]).columns.tolist()
categorical_columns = X.select_dtypes(include=['object']).columns.tolist()

numeric_columns, categorical_columns

(['age',
  'Medu',
  'Fedu',
  'traveltime',
  'studytime',
  'failures',
  'famrel',
  'freetime',
  'goout',
  'Dalc',
  'Walc',
  'health',
  'absences',
  'G1',
  'G2'],
 ['school',
  'sex',
  'address',
  'famsize',
  'Pstatus',
  'Mjob',
  'Fjob',
  'reason',
  'guardian',
  'schoolsup',
  'famsup',
  'paid',
  'activities',
  'nursery',
  'higher',
  'internet',
  'romantic'])

 2. Проверить пропуски в данных.

In [10]:
data.isna().sum()

school        0
sex           0
age           0
address       0
famsize       0
Pstatus       0
Medu          0
Fedu          0
Mjob          0
Fjob          0
reason        0
guardian      0
traveltime    0
studytime     0
failures      0
schoolsup     0
famsup        0
paid          0
activities    0
nursery       0
higher        0
internet      0
romantic      0
famrel        0
freetime      0
goout         0
Dalc          0
Walc          0
health        0
absences      0
G1            0
G2            0
G3            0
dtype: int64

3. Отнормировать численные переменные (`StandardScaler`, `MinMaxScaler`).

In [11]:
# Инициализация скейлеров
scaler_standard = StandardScaler()
#scaler_minmax = MinMaxScaler()

# Предполагаем, что `numeric_columns` содержит имена всех числовых столбцов
X_train_scaled_standard = scaler_standard.fit_transform(X_train[numeric_columns])
X_test_scaled_standard = scaler_standard.fit_transform(X_test[numeric_columns])

#X_train_scaled_minmax = scaler_minmax.fit_transform(X_train[numeric_columns])
#X_test_scaled_minmax = scaler_minmax.fit_transform(X_test[numeric_columns])

# Заменяем оригинальные числовые столбцы на отмасштабированные
X_train[numeric_columns] = X_train_scaled_standard
X_test[numeric_columns] = X_test_scaled_standard

 4. Закодировать категориальные признаки по одной из стратегий.

In [12]:
# Предполагаем, что `categorical_columns` содержит имена всех категориальных столбцов
encoder = OneHotEncoder(sparse_output=False)

# Обучаем энкодер на тренировочных данных и преобразуем тренировочные и тестовые данные
X_train_encoded = encoder.fit_transform(X_train[categorical_columns])
X_test_encoded = encoder.fit_transform(X_test[categorical_columns])

# OneHotEncoder возвращает массив NumPy, поэтому преобразуем его обратно в DataFrame
encoded_columns = list(encoder.get_feature_names_out(categorical_columns))  # Получаем новые имена столбцов
X_train_encoded_df = pd.DataFrame(X_train_encoded, index=X_train.index, columns=encoded_columns)
X_test_encoded_df = pd.DataFrame(X_test_encoded, index=X_test.index, columns=encoded_columns)

# Удаляем оригинальные категориальные столбцы и добавляем закодированные
X_train_final = X_train.drop(columns=categorical_columns).join(X_train_encoded_df)
X_test_final = X_test.drop(columns=categorical_columns).join(X_test_encoded_df)

In [13]:
X_train_final, X_test_final

(          age      Medu      Fedu  traveltime  studytime  failures    famrel  \
 181 -0.586396  0.246437  0.423207   -0.625779  -0.056853 -0.456744  0.064432   
 194 -0.586396 -0.680636  0.423207    0.828236  -1.254553 -0.456744  1.195575   
 173 -0.586396 -1.607709  0.423207   -0.625779  -0.056853  3.628097  0.064432   
 63  -0.586396  1.173510  0.423207   -0.625779   1.140847 -0.456744 -1.066710   
 253 -0.586396 -0.680636 -1.434203    0.828236  -1.254553 -0.456744 -1.066710   
 ..        ...       ...       ...         ...        ...       ...       ...   
 71  -1.374912  1.173510 -0.505498   -0.625779   2.338548 -0.456744 -1.066710   
 106 -1.374912 -0.680636 -0.505498   -0.625779   2.338548 -0.456744  1.195575   
 270  1.779151  0.246437  0.423207   -0.625779  -0.056853  2.266484  0.064432   
 348  0.202119  1.173510  0.423207   -0.625779   1.140847 -0.456744  0.064432   
 102 -1.374912  1.173510  1.351912   -0.625779  -1.254553 -0.456744  1.195575   
 
      freetime     goout  

## 4. Обучение на тренировочном множестве

 1. Линейная модель

In [14]:
linear_model = LinearRegression()
linear_model.fit(X_train_final, y_train)

LinearRegression()

2. Деревянная модель 

In [15]:
dt_regressor = DecisionTreeRegressor(max_depth=3) # Пример с глубиной 3
dt_regressor.fit(X_train_final, y_train)

DecisionTreeRegressor(max_depth=3)

3. K-ближайших соседей

In [16]:
knn_regressor = KNeighborsRegressor(n_neighbors=5) # Пример с 5 соседями
knn_regressor.fit(X_train_final, y_train)

KNeighborsRegressor()

 4. Случайный лес

In [17]:
rf_regressor = RandomForestRegressor(n_estimators=100, max_depth=3) # Пример с 100 деревьями и глубиной 3
rf_regressor.fit(X_train_final, y_train)

RandomForestRegressor(max_depth=3)

## 5. Подсчет метрик на train и test множествах

In [19]:
y_pred_linear = linear_model.predict(X_test_final)
print("MAE of Linear Regression:", mean_absolute_error(y_test, y_pred_linear))

y_pred_dt_regressor = dt_regressor.predict(X_test_final)
print("MAE of Decision Tree Regressor:", mean_absolute_error(y_test, y_pred_dt_regressor))

y_pred_knn_regressor = knn_regressor.predict(X_test_final)
print("MAE of KNN Regressor:", mean_absolute_error(y_test, y_pred_knn_regressor))

y_pred_rf_regressor = rf_regressor.predict(X_test_final)
print("MAE of Random Forest Regressor:", mean_absolute_error(y_test, y_pred_rf_regressor))

MAE of Linear Regression: 1.7464985907832278
MAE of Decision Tree Regressor: 1.3099908309101178
MAE of KNN Regressor: 2.2759493670886073
MAE of Random Forest Regressor: 1.1779405829123513


In [20]:
y_pred_linear = linear_model.predict(X_test_final)
print("RMSE of Linear Regression:", np.sqrt(mean_squared_error(y_test, y_pred_linear)))

y_pred_dt_regressor = dt_regressor.predict(X_test_final)
print("RMSE of Decision Tree Regressor:", np.sqrt(mean_squared_error(y_test, y_pred_dt_regressor)))

y_pred_knn_regressor = knn_regressor.predict(X_test_final)
print("RMSE of KNN Regressor:", np.sqrt(mean_squared_error(y_test, y_pred_knn_regressor)))

y_pred_rf_regressor = rf_regressor.predict(X_test_final)
print("RMSE of Random Forest Regressor:", np.sqrt(mean_squared_error(y_test, y_pred_rf_regressor)))

RMSE of Linear Regression: 2.4211311455022613
RMSE of Decision Tree Regressor: 2.168715787955683
RMSE of KNN Regressor: 3.0915974323226987
RMSE of Random Forest Regressor: 1.8619055084728435


In [21]:
y_pred_linear = linear_model.predict(X_test_final)
print("MAPE of Linear Regression:", mean_absolute_percentage_error(y_test, y_pred_linear))

y_pred_dt_regressor = dt_regressor.predict(X_test_final)
print("MAPE of Decision Tree Regressor:", mean_absolute_percentage_error(y_test, y_pred_dt_regressor))

y_pred_knn_regressor = knn_regressor.predict(X_test_final)
print("MAPE of KNN Regressor:", mean_absolute_percentage_error(y_test, y_pred_knn_regressor))

y_pred_rf_regressor = rf_regressor.predict(X_test_final)
print("MAPE of Random Forest Regressor:", mean_absolute_percentage_error(y_test, y_pred_rf_regressor))

MAPE of Linear Regression: 1843769656830703.8
MAPE of Decision Tree Regressor: 1131381405902227.8
MAPE of KNN Regressor: 1983864139651813.8
MAPE of Random Forest Regressor: 1223119390995983.5


In [22]:
data[data['G3'] == 0]

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
128,GP,M,18,R,GT3,T,2,2,services,other,...,3,3,3,1,2,4,0,7,4,0
130,GP,F,15,R,GT3,T,3,4,services,teacher,...,4,2,2,2,2,5,0,12,0,0
131,GP,F,15,U,GT3,T,1,1,at_home,other,...,4,3,3,1,2,4,0,8,0,0
134,GP,M,15,R,GT3,T,3,4,at_home,teacher,...,5,3,3,1,1,5,0,9,0,0
135,GP,F,15,U,GT3,T,4,4,services,at_home,...,4,3,3,1,1,5,0,11,0,0
136,GP,M,17,R,GT3,T,3,4,at_home,other,...,5,4,5,2,4,5,0,10,0,0
137,GP,F,16,U,GT3,A,3,3,other,other,...,4,3,2,1,1,5,0,4,0,0
140,GP,M,15,U,GT3,T,4,3,teacher,services,...,2,2,2,1,1,3,0,7,9,0
144,GP,M,17,U,GT3,T,2,1,other,other,...,5,4,5,1,2,5,0,5,0,0
146,GP,F,15,U,GT3,T,3,2,health,services,...,3,3,2,1,1,3,0,6,7,0


## 6. Сравнение метрик относительно train/test

Модель линейной регресси справилась лучше с данной задачей, исходя из вычисленных нами метрик. Таких как - MAE, RMSE и MAPE. Однако метрика MAPE не несет в себе полезной информации, так как наша целевая переменная распределена от 0 до 20 (если есть нулевые значения, то MAPE будет высокой). Для линейной модели метрики MAE и RMSE имеют наименьшие значения. Это указывает на то, что линейная регрессия в среднем дает более точные прогнозы и меньше ошибается в своих предсказаниях.

1. Какая модель справилась лучше с поставленной задачей?
Рандомный лес
2. Имеет ли место переобучение?
Нет
3. Имеет ли место недообучение?
Нет
4. Как можно улучшить метрики моделей?
 Использование кросс-валидации
 Удаление выбросов
 Более тщательное исследование данных
 Инженерия признаков (создание новых колонок, обьединение колонок)